# 1. Preamboli

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from mlflow.tracking import MlflowClient

import os
import re

import platform
import sys
import json

## Utils

In [ ]:
def save_dataset_version(dataset_name: str, df: pd.DataFrame):
    # Percorso della cartella dataset
    base_dir = "datasets"
    dataset_dir = os.path.join(base_dir, dataset_name)
    
    # Crea la cartella se non esiste
    os.makedirs(dataset_dir, exist_ok=True)
    
    # Trova i file già presenti che matchano lo schema vXX.csv
    existing_files = [f for f in os.listdir(dataset_dir) if re.match(r"v\d{2}\.csv", f)]
    
    if not existing_files:
        # Se non ci sono file, la prima versione è v01
        version_number = 1
    else:
        # Estrai i numeri delle versioni dai file
        versions = [int(re.findall(r"\d{2}", f)[0]) for f in existing_files]
        version_number = max(versions) + 1
    
    # Nome del file da salvare
    filename = f"v{version_number:02d}.csv"
    filepath = os.path.join(dataset_dir, filename)
    
    # Salva il dataframe
    df.to_csv(filepath, index=False)
    
    return filepath, version_number

# 2. Import e versioning del dataset

In [ ]:
dataset_name = "iris_dataset"

In [ ]:
# Carichiamo il dataset
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

In [ ]:
# Salviamo il dataset versionato localmente (puoi anche usare DVC o Git LFS)
dataset_path, dataset_version = save_dataset_version(dataset_name, df)

print(f"Dataset salvato in: {dataset_path}")
print(f"Versione del dataset: {dataset_version}")

df.head(2)

In [ ]:
# Split train/test
X = df[iris.feature_names]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# 3. Setup MLflow Tracking

In [ ]:
# Impostiamo il nome dell'esperimento
mlflow.set_experiment("Iris_Classification")

In [ ]:
# Funzione helper per loggare esperimenti
def train_and_log_model(model_name="IrisClassifier",
                        model_dict = {
                            "model": "RandomForest",
                            "n_estimators": 100,
                            "max_depth": None
                        },
                        dataset_version=None):
    
    with mlflow.start_run() as run:

        if model_dict["model"] == "RandomForest":
            # 1. Crea il modello
            model = RandomForestClassifier(n_estimators=model_dict["n_estimators"], max_depth=model_dict["max_depth"], random_state=42)
            model.fit(X_train, y_train)

            mlflow.log_param("n_estimators", model_dict["n_estimators"])
            mlflow.log_param("max_depth", model_dict["max_depth"])

            # # Log feature importance (se utile per analisi)
            # feature_importances = dict(zip(X_train.columns, model.feature_importances_))
            # mlflow.log_dict(feature_importances, "feature_importances.json")

        elif model_dict["model"] == "LogisticRegression":
            model = LogisticRegression(max_iter=model_dict["max_iter"], random_state=42)
            model.fit(X_train, y_train)

            mlflow.log_param("max_iter", model_dict["max_iter"])


        # 2. Previsioni e metriche
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)
        prec = precision_score(y_test, preds, average="weighted")
        rec = recall_score(y_test, preds, average="weighted")
        f1 = f1_score(y_test, preds, average="weighted")

        # 3. Log parametri principali del modello
        mlflow.log_param("model_class", model_dict["model"])
        if dataset_version is not None:
            mlflow.log_param("dataset_version", dataset_version)

        # 4. Log metriche
        mlflow.log_metric("accuracy", round( float(acc), 2))
        mlflow.log_metric("precision_weighted", round( float(prec), 2))
        mlflow.log_metric("recall_weighted", round( float(rec), 2))
        mlflow.log_metric("f1_weighted", round( float(f1), 2))

        # 5. Log del modello e dell’ambiente
        mlflow.sklearn.log_model(model, "model", registered_model_name=model_name)
        # mlflow.log_dict({
        #     "python_version": sys.version,
        #     "platform": platform.platform(),
        #     "mlflow_version": mlflow.__version__,
        #     "sklearn_version": model.__module__.split('.')[0]
        # }, "environment_info.json")

        # N.B. Non per forza tutti i modelli vanno registrati. 
        # Volendo, si può decidere di non registrare i modelli dentro questo codice, ma decidere a posteriori quali registrare e quali no

        # 6. Log di due righe di esempio dal dataset
        sample_input = X_train.head(2).copy()
        sample_input["target"] = y_train.iloc[:2].values
        mlflow.log_table(sample_input, "sample_input.parquet")

        # 7. Stampa riassunto
        print(f"Run {run.info.run_id} - Acc: {acc:.4f}")
        
        return run.info.run_id

## Runs

In [ ]:
n_estimators = 100 # 50 # 100
max_depth = 5 # 3 # 5

In [ ]:
D_randomforest = {"model": "RandomForest",
    "n_estimators": 50, # 100 , 50
    "max_depth": 5    # 3 , 5
    }

D_logistic = {
    "model": "LogisticRegression",
    "max_iter": 100 # 1000, 500
    }

In [ ]:
# Eseguiamo alcuni esperimenti con parametri diversi
run_id = train_and_log_model(   model_name = "Iris_Classifier",
                                model_dict = D_randomforest,
                                dataset_version = dataset_version,
                                )

# 4. Gestione dei modelli e MLflow Model Registry

In [ ]:
# Nome del modello registrato
MODEL_NAME = "Iris_Classifier"

client = MlflowClient()

# 1. Prendi tutte le versioni registrate del modello
versions = client.search_model_versions(f"name='{MODEL_NAME}'")

# 2. Trova l'accuracy migliore tra i run
best_run_id = None
best_accuracy = -1.0
best_model_version = None

for v in versions:
    run_id = v.run_id
    metrics = client.get_run(run_id).data.metrics
    acc = metrics.get("accuracy", None)
    if acc is not None and acc > best_accuracy:
        best_accuracy = acc
        best_run_id = run_id
        best_model_version = v.version

print(f"Miglior modello trovato: run_id={best_run_id}, versione={best_model_version}, accuracy={best_accuracy:.4f}")

In [ ]:
from mlflow.entities import Metric


In [ ]:
def metric_to_dict(metric):
    return {
        "key": metric.key,
        "value": metric.value,
        "timestamp": metric.timestamp,
        "step": metric.step
    }

In [ ]:
from mlflow.exceptions import MlflowException
import time

# Inizializza il client
client = MlflowClient()

# Nome del modello registrato e versione che vuoi modificare
model_name = "Iris_Classifier"
model_version = 1  # la versione che vuoi promuovere o spostare

# Numero massimo di tentativi
max_retries = 3
retry_delay = 2  # secondi

for attempt in range(max_retries):
    try:
        # Sposta il modello in staging
        client.transition_model_version_stage(
            name=model_name,
            version=model_version,
            stage="Staging",   # opzioni: "None", "Staging", "Production", "Archived"
            archive_existing_versions=False  # se True, sposta automaticamente le versioni esistenti dalla stessa fase in Archived
        )
        print(f"Modello {model_name} versione {model_version} spostato in Staging con successo")
        break  # Esci dal ciclo se l'operazione ha successo
        
    except MlflowException as e:
        print(f"Tentativo {attempt + 1} fallito: {e}")
        
        # Se non è l'ultimo tentativo, aspetta prima di riprovare
        if attempt < max_retries - 1:
            print(f"Riprovo tra {retry_delay} secondi...")
            time.sleep(retry_delay)
        else:
            print("Numero massimo di tentativi raggiunto. Operazione fallita.")
            # Qui puoi aggiungere ulteriori azioni di fallback se necessario
            
    except Exception as e:
        print(f"Errore imprevisto: {e}")
        break  # Interrompi per errori non previsti

In [ ]:
# # 3. Aggiorna lo stato del modello migliore (es. in Production)
# if best_model_version is not None:
#     client.transition_model_version_stage(
#         name=MODEL_NAME,
#         version=best_model_version,
#         stage="Staging",   # oppure "Staging"
#         archive_existing_versions=True  # sposta gli altri modelli fuori da Production
#     )
#     print(f"Il modello versione {best_model_version} è stato promosso a Production.")